In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from warnings import filterwarnings, simplefilter
filterwarnings('ignore')
simplefilter('ignore')

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv', parse_dates = ['time'])
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', parse_dates = ['time'])
sample = pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')

In [ ]:
train = train.drop('row_id', axis = 1)
test = test.drop('row_id', axis = 1)

In [ ]:
display(train, test)

In [ ]:
train = train.loc[train['time'] > '1991-05-30']
train['time'] = train['time'].astype(int)
test['time'] = test['time'].astype(int)

In [ ]:
mx = MinMaxScaler()
train['time'] = mx.fit_transform(train['time'].to_frame())
test['time'] = mx.transform(test['time'].to_frame())

In [ ]:
display(train, test)

In [ ]:
le = LabelEncoder()
train['direction'] = le.fit_transform(train['direction'])
test['direction'] = le.transform(test['direction'])

In [ ]:
display(train, test)

In [ ]:
y = train.pop('congestion')
display(train, y)

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(train, y, test_size = 0.2, shuffle = False)

In [ ]:
model = RandomForestRegressor(random_state = 5, n_jobs = -1, max_depth = 10).fit(xtrain, ytrain)

In [ ]:
yhat = model.predict(xval)
print(f'MAE : {mean_absolute_error(yval, yhat)}')

In [ ]:
model = RandomForestRegressor(random_state = 5, n_jobs = -1, max_depth = 10).fit(train, y)

In [ ]:
ypred = model.predict(test)
print(ypred)

In [ ]:
sample

In [ ]:
sample['congestion'] = ypred
sample.to_csv('submission.csv', index = False)
display(sample)